# ImageNet A/B/E Benchmark (Colab)

This notebook prepares ImageNet-1k (official route), then runs A/B/E param-matched benchmarks using paper-style training flags.

Prerequisites:
- You have access to ILSVRC2012 (ImageNet-1k) and placed these files in Google Drive:
  - `ILSVRC2012_img_train.tar`
  - `ILSVRC2012_img_val.tar`
  - `ILSVRC2012_devkit_t12.tar`



In [ ]:
# Clone and install
pip -q install -U pip
if [ ! -d MoP ]; then git clone https://github.com/Eran-BA/MoP.git ; fi
cd MoP || exit 1
pip -q install -r requirements.txt


In [ ]:
# Mount Drive and configure paths
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
IMNET_SRC = '/content/drive/MyDrive/imagenet'  # folder holding the three tar files
IMNET_ROOT = '/content/imagenet'               # extraction target
print('Source:', IMNET_SRC, '\nTarget:', IMNET_ROOT)
os.makedirs(IMNET_ROOT, exist_ok=True)


In [ ]:
# Verify tar files
req = ['ILSVRC2012_img_train.tar','ILSVRC2012_img_val.tar','ILSVRC2012_devkit_t12.tar']
missing = [f for f in req if not os.path.exists(os.path.join(IMNET_SRC,f))]
assert not missing, f'Missing: {missing} in {IMNET_SRC}'


In [ ]:
# Extract train/val/devkit
echo "Extracting train..."
mkdir -p /content/imagenet/train /content/imagenet/val /content/imagenet/devkit

tar -xvf /content/drive/MyDrive/imagenet/ILSVRC2012_img_train.tar -C /content/imagenet/train
cd /content/imagenet/train && find . -name "*.tar" -exec sh -c 'd=$(basename "$1" .tar); mkdir -p "$d"; tar -xf "$1" -C "$d"; rm "$1"' _ {} \;

cd /content/MoP || true
tar -xvf /content/drive/MyDrive/imagenet/ILSVRC2012_img_val.tar -C /content/imagenet/val

tar -xvf /content/drive/MyDrive/imagenet/ILSVRC2012_devkit_t12.tar -C /content/imagenet/devkit


In [ ]:
# Organize validation set into class folders
import os, shutil
import scipy.io as sio

devkit = '/content/imagenet/devkit/ILSVRC2012_devkit_t12'
meta = sio.loadmat(os.path.join(devkit,'data','meta.mat'))['synsets']
# Build mapping: ILSVRC2012_ID -> WNID
id2wnid = {}
for entry in meta:
    ILSVRC2012_ID = int(entry['ILSVRC2012_ID'][0][0]) if entry['ILSVRC2012_ID'].size else 0
    if ILSVRC2012_ID>0:
        wnid = str(entry['WNID'][0])
        id2wnid[ILSVRC2012_ID] = wnid

with open(os.path.join(devkit,'data','ILSVRC2012_validation_ground_truth.txt'),'r') as f:
    gt = [int(x.strip()) for x in f if x.strip()]

val_dir = '/content/imagenet/val'
imgs = sorted([x for x in os.listdir(val_dir) if x.lower().endswith(('.jpeg','.jpg'))])
assert len(imgs)==len(gt), f'mismatch: {len(imgs)} images vs {len(gt)} labels'

for img, cls_id in zip(imgs, gt):
    wnid = id2wnid[cls_id]
    dst = os.path.join(val_dir, wnid)
    os.makedirs(dst, exist_ok=True)
    shutil.move(os.path.join(val_dir,img), os.path.join(dst,img))


In [ ]:
# Run A/B/E for ViT-B/16 (~86M). Reduce batch if OOM.
export IMAGENET_ROOT=/content/imagenet
python experiments/imagenet_ab_param_budgets.py \
  --data_root $IMAGENET_ROOT \
  --targets 86000000 \
  --models A B E \
  --img_size 224 --patch 16 \
  --steps 90000 --eval_every 1000 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 \
  --use_randaug --randaug_n 2 --randaug_m 9 --random_erasing 0.25 \
  --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0


In [ ]:
# Optional: ViT-L/16 and ViT-H/14
python experiments/imagenet_ab_param_budgets.py --data_root $IMAGENET_ROOT \
  --targets 307000000 --models A B E --img_size 224 --patch 16 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 --use_randaug --randaug_n 2 --randaug_m 9 \
  --random_erasing 0.25 --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0

python experiments/imagenet_ab_param_budgets.py --data_root $IMAGENET_ROOT \
  --targets 632000000 --models A B E --img_size 224 --patch 14 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 --use_randaug --randaug_n 2 --randaug_m 9 \
  --random_erasing 0.25 --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0


In [ ]:
# Generate paper tables
python experiments/ab5_paper_benchmark.py
ls -la results/paper_benchmark
